# Part A --Intrinsic and Extrinsic Parameter

![title](hw1_2.jpg)
![title](hw1_1.jpg)

# Steps shown below: Computed Using Python

In [19]:
import numpy as np

import pytransform3d.rotations as pr
from pytransform3d.plot_utils import plot_vector

In [20]:
origin = np.array([0, 0, 0])

# basis vectors
x = np.array([1, 0, 0])
y = np.array([0, 1, 0])
z = np.array([0, 0, 1])

# basis vectors as homogeneous coordinates
xh = np.array([1, 0, 0, 1])
yh = np.array([0, 1, 0, 1])
zh = np.array([0, 0, 1, 1])

In [21]:
# Get transformation matrix
def get_rot_x(angle):
    '''
    transformation matrix that rotates a point about the standard X axis
    '''
    Rx = np.zeros(shape=(3, 3))
    Rx[0, 0] = 1
    Rx[1, 1] = np.cos(angle)
    Rx[1, 2] = -np.sin(angle)
    Rx[2, 1] = np.sin(angle)
    Rx[2, 2] = np.cos(angle)
    
    return Rx

def get_rot_y(angle):
    Ry = np.zeros(shape=(3, 3))
    Ry[0, 0] = np.cos(angle)
    Ry[0, 2] = -np.sin(angle)
    Ry[2, 0] = np.sin(angle)
    Ry[2, 2] = np.cos(angle)
    Ry[1, 1] = 1
    
    return Ry

def get_rot_z(angle):
    Rz = np.zeros(shape=(3, 3))
    Rz[0, 0] = np.cos(angle)
    Rz[0, 1] = -np.sin(angle)
    Rz[1, 0] = np.sin(angle)
    Rz[1, 1] = np.cos(angle)
    Rz[2, 2] = 1
    
    return Rz

In [22]:
def create_rotation_transformation_matrix(angles, order):
    fn_mapping = {'x': get_rot_x, 'y': get_rot_y, 'z': get_rot_z}
    net = np.identity(3)
    for angle, axis in list(zip(angles, order))[::-1]:
        if fn_mapping.get(axis) is None:
            raise ValueError("Invalid axis")
        R = fn_mapping.get(axis)
        net = np.matmul(net, R(angle))
        
    return net

def create_translation_matrix(offset):
    T = np.identity(4)
    T[:3, 3] = offset
    return T

In [23]:
R = create_rotation_transformation_matrix([np.pi/6], 'y')
R_ = np.identity(4)
R_[:3, :3] = R

# create translation transformation matrix
T_ = create_translation_matrix(np.array([0, 28, 0]))

In [24]:
R_, T_

(array([[ 0.8660254,  0.       , -0.5      ,  0.       ],
        [ 0.       ,  1.       ,  0.       ,  0.       ],
        [ 0.5      ,  0.       ,  0.8660254,  0.       ],
        [ 0.       ,  0.       ,  0.       ,  1.       ]]),
 array([[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0., 28.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]))

In [25]:
def compute_intrinsic_parameter_matrix(f, s, a, cx, cy):
    K = np.identity(3)
    K[0, 0] = f
    K[0, 1] = s
    K[0, 2] = cx
    K[1, 1] = a * f
    K[1, 2] = cy
    
    return K

In [26]:
intrinsic_matrix = compute_intrinsic_parameter_matrix(3.37,0,1,0,28)
intrinsic_matrix

array([[ 3.37,  0.  ,  0.  ],
       [ 0.  ,  3.37, 28.  ],
       [ 0.  ,  0.  ,  1.  ]])

In [11]:
import cv2

In [12]:
# Four corners in source image
pts_src = np.array([[684, 530], [728, 529], [685, 574],[728, 572]])

# Four corners in destination image
pts_dst = np.array([[661, 591],[704, 592],[660, 633],[702, 634]])

# Calculate Homography
h, status = cv2.findHomography(pts_src, pts_dst)

In [13]:
h

array([[ 4.30592128e-01, -2.07816394e-02,  2.00839446e+02],
       [-2.06908950e-01,  7.09575758e-01,  1.98508004e+02],
       [-4.04476587e-04,  1.77654352e-05,  1.00000000e+00]])

# Homography

## Defining image points


xs, ys - coordinates of  source image 

xd, ys - coordinates of destination image

In [28]:
xs = [684, 728, 686, 728]
ys = [530, 529, 574, 572]
xd = [661, 704, 660, 702]
yd = [590, 592, 633, 634]


$$\begin{bmatrix} xd \\ yd \\ 1 \end{bmatrix} \equiv  \begin{bmatrix} xd \\ yd \\ zd \end{bmatrix} = \begin{bmatrix} h_{11} & h_{12} & h_{13} \\ h_{21} & h_{22} & h_{23} \\ h_{31} & h_{32} & h_{33} \end{bmatrix} \begin{bmatrix} xs \\ ys \\ 1 \end{bmatrix}$$ 

Homogeneous Representation: $$ \frac{xd^{(i)}}{zd^{(i)}} and \frac{yd^{(i)}}{zd^{(i)}}$$

Goal: Solve: A.h=0

In [43]:
xdxs=-np.multiply(np.array(xd),np.array(xs))
xdys =-np.multiply(np.array(xd),np.array(ys)) 
ydxs=-np.multiply(np.array(yd),np.array(xs))
ydys=-np.multiply(np.array(yd),np.array(ys))
xdxs, xdys, ydxs, ydys

(array([-452124, -512512, -452760, -511056]),
 array([-350330, -372416, -378840, -401544]),
 array([-403560, -430976, -434238, -461552]),
 array([-312700, -313168, -363342, -362648]))

In [44]:
#Computing for all four points
A=[]
for i in range(0,4):
    mat1 = [xs[i],ys[i],1,0,0,0,xdxs[i],xdys[i],-xd[i]]
    mat2 = [0,0,0,xs[i],ys[i],1,ydxs[i],ydys[i],-yd[i]]
    A.append(mat1)
    A.append(mat2)
A = np.array(A)
A

array([[    684,     530,       1,       0,       0,       0, -452124,
        -350330,    -661],
       [      0,       0,       0,     684,     530,       1, -403560,
        -312700,    -590],
       [    728,     529,       1,       0,       0,       0, -512512,
        -372416,    -704],
       [      0,       0,       0,     728,     529,       1, -430976,
        -313168,    -592],
       [    686,     574,       1,       0,       0,       0, -452760,
        -378840,    -660],
       [      0,       0,       0,     686,     574,       1, -434238,
        -363342,    -633],
       [    728,     572,       1,       0,       0,       0, -511056,
        -401544,    -702],
       [      0,       0,       0,     728,     572,       1, -461552,
        -362648,    -634]])

# Solving for H

In [45]:
inner_product = np.dot(A.T,A)
w, v = np.linalg.eig(inner_product)
v = v.T 
v_prime = v[-1]
h = v_prime.reshape((3,3))
print('Homography:', h)

Homography: [[ 2.95783913e-03 -1.23633536e-03  7.26806249e-01]
 [ 9.84941133e-05  2.10475926e-03  6.86819450e-01]
 [-1.93019222e-07 -1.55165163e-06  4.12340350e-03]]
